In [91]:
import numpy
import pandas
import sklearn.cluster
import scipy.stats
from matplotlib import pyplot as plt



In [65]:
def get_features_names():
    with open('./data/NHANES/features.txt') as file:
        return [ f for f in file.read().split('\n') if f]
    
def load_data(key_col = 'DR1IFDCD'):
    df = pandas.read_csv('./data/NHANES/nhanes-dietary-complete-2011.csv')
    df.index = df[key_col]
    df = df.rename({'DR1IGRMS': 'weight'}, axis='columns')
    return df

data = load_data()
print('raw data', data.shape)
data.head(1)

raw data (31499, 85)


,SEQN,WTDRD1,WTDR2D,DR1ILINE,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,...,DR1IM181,DR1IM201,DR1IM221,DR1IP182,DR1IP183,DR1IP184,DR1IP204,DR1IP205,DR1IP225,DR1IP226
DR1IFDCD,,,,,,,,,,,,,,,,,,,,,
94000100,62161,58373.37572,46955.91202,1,1,63,2,2,9.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
def clean_data(df):
    feature_names = get_features_names() + ['weight']
    df = df[df['weight'] >= 1e-3] # only non-zero weight
    df = df[feature_names] # only nutritional
    df = df.dropna(axis='rows') # only with valid entries everywhere
    return df

cleaned = clean_data(data)
print('cleaned', cleaned.shape)

cleaned (31303, 66)


In [122]:
def meanTop(df, top=5):
    return df.sort_values('weight').head(top).mean()

def get_features(df):
    df = df.groupby(df.index).apply(meanTop) # aggregate to one entry per item
    weight = df.weight
    df = df.drop('weight', axis='columns')    
    df = df.divide(weight, axis='rows') # scale by weight    
    zz = scipy.stats.zscore(df, axis=0) # z-score to account for log-normal values
    return zz
    

features = get_features(cleaned)
print('features', features.shape)

(3104, 65) [[-1.01485507 -0.57066366 -0.75107337 -0.26915831 -0.66343084]
 [-0.86488523 -0.58401554 -0.7555468  -0.27267361 -0.66343084]
 [-1.11380986 -0.55557559 -0.75144944 -0.28534147 -0.66343084]]
features (3104, 65)


In [128]:
epsilon = 1
min_pts = 4
est = sklearn.cluster.DBSCAN(eps=epsilon, min_samples=min_pts, metric='euclidean')
clusters = est.fit_predict(features)

In [170]:
def interclass_distances(X, clusters, metric='euclidean', agg='min'):
    if type(agg) == str:
        agg = getattr(numpy, agg)
    
    from sklearn.metrics import pairwise_distances
    n_clusters = numpy.max(clusters)
    dist = numpy.zeros((n_clusters, n_clusters))
    for i in range(n_clusters):
        for j in range(n_clusters):
            A = X[clusters == i]
            B = X[clusters == j]
            p = pairwise_distances(A, B, metric=metric)
            dist[i, j] = agg(p)
    
    return dist

def min_interclass_distance(X, clusters):
    dist = interclass_distances(features, clusters, agg='min')
    mask = numpy.ones(dist.shape, dtype=bool)
    numpy.fill_diagonal(mask, 0)
    min_value = numpy.min(dist[mask], axis=0)
    #min_value = dist[mask].min(axis=0)
    return min_value


import seaborn

dist = min_interclass_distance(features, clusters)
dist
#seaborn.heatmap(dist)

0.7538939042811336

In [138]:
numpy.max(clusters)

67

In [130]:
len(clusters[clusters == -1])

2095